In [7]:
from vowpalwabbit import pyvw
from datetime import datetime

In [15]:
vw = pyvw.vw('--cb 3 -P 1')
vw.get_enabled_reductions()

['gd', 'scorer', 'csoaa_ldf', 'cb_adf', 'shared_feature_merger', 'cb_to_cbadf']

In [16]:
import tensorboardX as tx

In [31]:
logdir = "logs/scalars/" + datetime.now().strftime("%Y%m%d-%H%M%S")  # logs directory
file_writer = tx.SummaryWriter(logdir+'/iris', flush_secs=5)   # creating file writer

In [6]:
file_writer.add_graph(vw)

/home/muhammadammarabid/anaconda3/lib/python3.8/site-packages/setuptools/distutils_patch.py:25: UserWarning: Distutils was imported before Setuptools. This usage is discouraged and may exhibit undesirable behaviors or errors. Please use Setuptools' objects directly or at least import Setuptools first.
  warnings.warn(


AttributeError: 'vw' object has no attribute 'training'

In [59]:

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
from tensorboardX import SummaryWriter

dummy_input = (torch.zeros(1, 3),)


class LinearInLinear(nn.Module):
    def __init__(self):
        super(LinearInLinear, self).__init__()
        self.l = nn.Linear(3, 5)

    def forward(self, x):
        return self.l(x)

with SummaryWriter(comment='LinearInLinear') as w:
    w.add_graph(LinearInLinear(), dummy_input, True)    

graph(%self.1 : __torch__.___torch_mangle_3.LinearInLinear,
      %x : Float(1, 3, strides=[3, 1], requires_grad=0, device=cpu)):
  %13 : __torch__.torch.nn.modules.linear.___torch_mangle_2.Linear = prim::GetAttr[name="l"](%self.1)
  %16 : Tensor = prim::GetAttr[name="bias"](%13)
  %17 : Tensor = prim::GetAttr[name="weight"](%13)
  %18 : Float(1, 5, strides=[5, 1], requires_grad=1, device=cpu) = aten::linear(%x, %17, %16), scope: __module.l # /home/muhammadammarabid/anaconda3/lib/python3.8/site-packages/torch/nn/functional.py:1847:0
  return (%18)



In [11]:
import tensorflow as tf

In [16]:
help(tf.Graph)

Help on class Graph in module tensorflow.python.framework.ops:

class Graph(builtins.object)
 |  A TensorFlow computation, represented as a dataflow graph.
 |  
 |  Graphs are used by `tf.function`s to represent the function's computations.
 |  Each graph contains a set of `tf.Operation` objects, which represent units of
 |  computation; and `tf.Tensor` objects, which represent the units of data that
 |  flow between operations.
 |  
 |  ### Using graphs directly (deprecated)
 |  
 |  A `tf.Graph` can be constructed and used directly without a `tf.function`, as
 |  was required in TensorFlow 1, but this is deprecated and it is recommended to
 |  use a `tf.function` instead. If a graph is directly used, other deprecated
 |  TensorFlow 1 classes are also required to execute the graph, such as a
 |  `tf.compat.v1.Session`.
 |  
 |  A default graph can be registered with the `tf.Graph.as_default` context
 |  manager. Then, operations will be added to the graph instead of being executed
 | 

In [23]:
from tensorboardX.proto.graph_pb2 import GraphDef
from tensorboardX.proto.node_def_pb2 import NodeDef
from tensorboardX.proto import event_pb2

In [32]:
nodes = []
reductions = vw.get_enabled_reductions()

for ind, layer in enumerate(reductions):
    if ind > 0:
        nodes.append(NodeDef(name=layer, op='op', input=[reductions[ind-1]]))
    else:
        nodes.append(NodeDef(name=layer, op='op'))
g = GraphDef(node=nodes)

event = event_pb2.Event(graph_def=g.SerializeToString())
file_writer._get_file_writer().add_event(event)


In [26]:
file_writer._get_file_writer().flush()
# file_writer.add_scalar('ss', 5000, 0)
# file_writer.add_scalar('ss', 50, 1)

print(reductions)
print(g)

['gd', 'scorer', 'csoaa_ldf', 'cb_adf', 'shared_feature_merger', 'cb_to_cbadf']
node {
  name: "gd"
  op: "op"
}
node {
  name: "scorer"
  op: "op"
  input: "gd"
}
node {
  name: "csoaa_ldf"
  op: "op"
  input: "scorer"
}
node {
  name: "cb_adf"
  op: "op"
  input: "csoaa_ldf"
}
node {
  name: "shared_feature_merger"
  op: "op"
  input: "cb_adf"
}
node {
  name: "cb_to_cbadf"
  op: "op"
  input: "shared_feature_merger"
}



In [9]:
help(GraphDef.__init__)

Help on wrapper_descriptor:

__init__(self, /, *args, **kwargs)
    Initialize self.  See help(type(self)) for accurate signature.



In [8]:
help(NodeDef)

Help on class NodeDef in module tensorboardX.proto.node_def_pb2:

class NodeDef(google.protobuf.pyext._message.CMessage, google.protobuf.message.Message)
 |  A ProtocolMessage
 |  
 |  Method resolution order:
 |      NodeDef
 |      google.protobuf.pyext._message.CMessage
 |      google.protobuf.message.Message
 |      builtins.object
 |  
 |  Data descriptors defined here:
 |  
 |  attr
 |      Field tensorboardX.NodeDef.attr
 |  
 |  device
 |      Field tensorboardX.NodeDef.device
 |  
 |  input
 |      Field tensorboardX.NodeDef.input
 |  
 |  name
 |      Field tensorboardX.NodeDef.name
 |  
 |  op
 |      Field tensorboardX.NodeDef.op
 |  
 |  ----------------------------------------------------------------------
 |  Data and other attributes defined here:
 |  
 |  AttrEntry = <class 'tensorboardX.proto.node_def_pb2.AttrEntry'>
 |      A ProtocolMessage
 |  
 |  DESCRIPTOR = <google.protobuf.pyext._message.MessageDescriptor object>
 |  
 |  --------------------------------------

In [31]:
help(file_writer.add_onnx_graph)

Help on method add_onnx_graph in module tensorboardX.writer:

add_onnx_graph(onnx_model_file) method of tensorboardX.writer.SummaryWriter instance
    Add onnx graph to TensorBoard.
    
    Args:
        onnx_model_file (string): The path to the onnx model.

